In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!python --version

Python 3.6.9


In [3]:
# input 
path_vectors = "./en_vectors_web_lg/en_vectors_web_lg-2.1.0"
path_model = './model_lstm/' 
path_data = './Tripadvisor_data.csv'
res_id = 9  

# output
"""
return

main: main (dataframe)
main_pos: number of positive reviews (int) 
main_neg: number of negative reviews (int)

food_df: food (dataframe)
food_pos: number of positive reviews (int)
food_neg: number of negative reviews (int)

ser_df, service (dataframe)
ser_pos, number of positive reviews (int)
ser_neg, number of negative reviews (int)

env_df: environment (dataframe) 
env_pos: number of positive reviews (int)
env_neg: number of negative reviews (int)

price_df: price (dataframe)
price_pos: number of positive reviews (int)
price_neg: number of negative reviews (int)

overall_score: (int)
food_score: (int)
ser_score: (int)
env_score: (int)
price_score: (int)

foodtop_pos:  top 10 positive food review (Series)
foodtop_neg:  top 10 negative food review (Series)
sertop_pos:   top 10 positive service review (Series)
sertop_neg:   top 10 negative service review (Series)
envtop_pos:   top 10 positive environment review (Series)
envtop_neg:   top 10 negative environment review (Series)
pricetop_pos: top 10 positive price review (Series)
pricetop_neg: top 10 negative price review (Series)
"""

'\nreturn\n\nmain: main (dataframe)\nmain_pos: number of positive reviews (int) \nmain_neg: number of negative reviews (int)\n\nfood_df: food (dataframe)\nfood_pos: number of positive reviews (int)\nfood_neg: number of negative reviews (int)\n\nser_df, service (dataframe)\nser_pos, number of positive reviews (int)\nser_neg, number of negative reviews (int)\n\nenv_df: environment (dataframe) \nenv_pos: number of positive reviews (int)\nenv_neg: number of negative reviews (int)\n\nprice_df: price (dataframe)\nprice_pos: number of positive reviews (int)\nprice_neg: number of negative reviews (int)\n\noverall_score: (int)\nfood_score: (int)\nser_score: (int)\nenv_score: (int)\nprice_score: (int)\n\nfoodtop_pos:  top 10 positive food review (Series)\nfoodtop_neg:  top 10 negative food review (Series)\nsertop_pos:   top 10 positive service review (Series)\nsertop_neg:   top 10 negative service review (Series)\nenvtop_pos:   top 10 positive environment review (Series)\nenvtop_neg:   top 10 ne

In [4]:
!ls
%cd /content/drive/My Drive/Colab Notebooks/CA_NLP

drive  sample_data
/content/drive/My Drive/Colab Notebooks/CA_NLP


In [5]:
# if run in colab 
!pip install Keras==2.0.9
!pip install cytoolz==0.10.1

import plac
import random
import pathlib
import cytoolz
import numpy
import pandas as pd 
import keras
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
from spacy.compat import pickle
import spacy
!python -m spacy download en_core_web_sm 

     |████████████████████████████████| 307kB 12.6MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.9 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 481kB 12.5MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.10.1-cp36-cp36m-linux_x86_64.whl size=1256673 sha256=c9a2b1f366358f3a59e84336dc1b5af9acfee755340626e33743639e72ed7e29
  Stored in directory: /root/.cache/pip/wheels/64/2a/18/d962b614e055577e7d9a3e4813e0742f822ca9c8800cc3783a
Successfully built cytoolz


Using TensorFlow backend.


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
class SentimentAnalyser(object):
    @classmethod

    # def load(cls, path, nlp, max_length=100):
    def load(cls, nlp, path, max_length):
        with open(path+"config.json") as file_:
            model = model_from_json(file_.read())
        with open(path+"model",'rb') as file_:
            lstm_weights = pickle.load(file_)
        embeddings = nlp.vocab.vectors.data
        model.set_weights([embeddings] + lstm_weights)
        return cls(model, max_length=max_length)

    def __init__(self, model, max_length=100):
            self._model = model
            self.max_length = max_length

    def __call__(self, doc):
            X = get_features([doc], self.max_length)
            y = self._model.predict(X)
            self.set_sentiment(doc, y)

    def pipe(self, docs, batch_size=1000):
            def get_features(docs, max_length):
                    docs = list(docs)
                    Xs = numpy.zeros((len(docs), max_length), dtype="int32")
                    for i, doc in enumerate(docs):
                        j = 0
                        for token in doc:
                            vector_id = token.vocab.vectors.find(key=token.orth)
                            if vector_id >= 0:
                                Xs[i, j] = vector_id
                            else:
                                Xs[i, j] = 0
                            j += 1
                            if j >= max_length:
                                break
                    return Xs
            for minibatch in cytoolz.partition_all(batch_size, docs):
                minibatch = list(minibatch)
                sentences = []
                for doc in minibatch:
                    sentences.extend(doc.sents)
           
                Xs = get_features(sentences, self.max_length)
                ys = self._model.predict(Xs)
                for sent, label in zip(sentences, ys):
                    sent.doc.sentiment += label 
                for doc in minibatch:
                    yield doc



    def set_sentiment(self, doc, y):
            doc.sentiment = float(y[0])
            
    def get_labelled_sentences(docs, doc_labels):
        labels = []
        sentences = []
        for doc, y in zip(docs, doc_labels):
            for sent in doc.sents:
                sentences.append(sent)
                labels.append(y)
        return sentences, numpy.asarray(labels, dtype="int32")

    def compile_lstm(embeddings, shape, settings):
        model = Sequential()
        model.add(
            Embedding(
                embeddings.shape[0],
                embeddings.shape[1],
                input_length=shape["max_length"],
                trainable=False,
                weights=[embeddings],
                mask_zero=True,
            )
        )
        model.add(TimeDistributed(Dense(shape["nr_hidden"], use_bias=False)))
        model.add(
            Bidirectional(
                LSTM(
                    shape["nr_hidden"],
                    recurrent_dropout=settings["dropout"],
                    dropout=settings["dropout"],
                )
            )
        )
        model.add(Dense(shape["nr_class"], activation="sigmoid"))
        model.compile(
            optimizer=Adam(lr=settings["lr"]),
            loss="binary_crossentropy",
            metrics=["accuracy"],
        )
        return model


In [8]:
# for preprocess
nlp = spacy.load('en_core_web_sm',disable=['parser','ner'])
nlp.add_pipe(nlp.create_pipe("sentencizer"))

# for sentiment
nlp_sentiment = spacy.load(path_vectors)
nlp_sentiment.add_pipe(nlp.create_pipe("sentencizer"))
nlp_sentiment.add_pipe(SentimentAnalyser.load(nlp_sentiment, path_model, 100))

# check pipeline
print(nlp.pipeline)
print(nlp_sentiment.pipeline)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






[('tagger', <spacy.pipeline.pipes.Tagger object at 0x7f565deda710>), ('sentencizer', <spacy.pipeline.pipes.Sentencizer object at 0x7f565deda4e0>)]
[('sentencizer', <spacy.pipeline.pipes.Sentencizer object at 0x7f565deda400>), ('SentimentAnalyser', <__main__.SentimentAnalyser object at 0x7f5648ee1668>)]


# 0. Load

In [0]:
# input from res_id 

trip = pd.read_csv(path_data,
                   usecols=['res_id','review'],
                   header=0,
                   encoding='utf-8')

trip = trip[trip['res_id']==res_id]
trip['review'] = trip['review'].str.lower()

# 1. Main

In [10]:
def docs2sent(docs):

# pipeline can process with batches ,will be faster than pipeline single element 
  allsents =list()
  for docsents in list(nlp.pipe(list(docs['review']))):
    for doc in docsents.sents:
      allsents.append(doc.text)
  print('docs2sent:')
  print(len(docs))
  print(len(allsents))
  print(type(allsents[0])) 
  print(allsents[0])

  return allsents

# def set_sentiment(score):

#   if score >=0.5:
#     return 'positive'
#   else:
#     return 'negative'

# def set_polarity(score):

#   if score >=0.5:
#     return 1
#   else:
#     return 0

def num_polarity(df):

  num_pos = sum(df['polarity'])
  num_neg = len(df['polarity']) - num_pos
  print('number of positive reviews: ' + str(num_pos))
  print('number of negative reviews: ' + str(num_neg))

  return num_pos, num_neg

def sent2score(df):

  allsents = docs2sent(df)
  review =list()
  score = list()
  for doc in nlp_sentiment.pipe(allsents):
      review.append(doc.text)
      score.append(doc.sentiment)
  main = pd.DataFrame({'review':review,'score':score})
  main['polarity'] = main['score'].apply(lambda x: 1 if x >= 0.5 else 0)
  main['sentiment'] = main['score'].apply(lambda x: 'positive' if x >= 0.5 else 'negative')
  main_pos, main_neg = num_polarity(main)
  main.to_csv('main.csv')

  return main, main_pos, main_neg

# dashboard output 
main, main_pos, main_neg = sent2score(trip)

docs2sent:
1824
10380
<class 'str'>
beautiful decor, and though a relatively large restaurant, it's broken into smaller more intimate spaces, creating a very comforting ambiance.
number of positive reviews: 7933
number of negative reviews: 2447


# 2. categories 

|index|categories|review|socre|polarity|sentiment
|---|---|---|---|---|---
|1|food|this food is delicious|0.9|1|positive


In [11]:
def to_category(df, category_name):
  index = list()
  for idx, val in enumerate(df['review']):
    for token in nlp(val,disable=['sentencizer']):
        if token.pos_ in ['NOUN'] and token.text in category_name:
            index.append(idx)
  category = df.iloc[index].drop_duplicates()
  category.to_csv(f'{category_name[0]}.csv')
  ca_pos, ca_neg = num_polarity(category)
  return category, ca_pos, ca_neg

food_tag = ['food','menu','dishes','taste','course','wine','dish',
             'dessert','beef','cake','drinks','seafood','tea','steak','meat',
             'salad','desserts','chicken','prok','coffee','ingredients','pizza',
             'pasta','beer','soup','rice','bread']
ser_tag = ['service','staff','experience','attitude','table','team',
           'munute','munutes', 'chef','waiter','manager']
env_tag = ['environemnt','ambience','atmosphere']
price_tag = ['price','value','money','dollar']

# dashboard output 
food_df, food_pos, food_neg = to_category(main, food_tag)
ser_df, ser_pos, ser_neg = to_category(main, ser_tag)
env_df, env_pos, env_neg = to_category(main, env_tag)
price_df, price_pos, price_neg = to_category(main, price_tag)

number of positive reviews: 2464
number of negative reviews: 663
number of positive reviews: 2177
number of negative reviews: 432
number of positive reviews: 365
number of negative reviews: 14
number of positive reviews: 136
number of negative reviews: 74


In [12]:
food_df.head()

review  ...  sentiment
1                   then there's the food selection!!  ...   positive
2   a buffet with everything you can imagine: seaf...  ...   positive
3   we met chef sundar, who surprised us with a sp...  ...   positive
9   she even went through the trouble of getting m...  ...   negative
11  very impressive service and good food, thank y...  ...   positive

[5 rows x 4 columns]

# 2.1 summary

In [13]:
# overall sentiment score by categories

overall_score = (main_pos/(main_pos + main_neg))*10
print('overall_score',overall_score)
food_score = (food_pos/(food_pos+food_neg))*10
print('food score',food_score)
ser_score = (ser_pos/(ser_pos+ser_neg))*10
print('service score',ser_score)
env_score = (env_pos/(env_pos+env_neg))*10
print('environment score',env_score)
price_score = (price_pos/(price_pos+price_neg))*10
print('price score',price_score)

overall_score 7.642581888246628
food score 7.87975695554845
service score 8.34419317746263
environment score 9.630606860158311
price score 6.476190476190476


In [14]:
# top reviews , What people are saying about food, service, environment, price

def get_top(df):
  toppos = df.sort_values(by='score')[-1:-11:-1].reset_index().review
  topneg = df.sort_values(by='score',)[0:10].reset_index().review
    # set position
  print(topneg[:10])
  return toppos, topneg


print('What people are saying about food:')
foodtop_pos, foodtop_neg = get_top(food_df)
print('What people are saying about service:')
sertop_pos, sertop_neg = get_top(ser_df)
print('What people are saying about environment:')
envtop_pos, envtop_neg = get_top(env_df)
print('What people are saying about price:')
pricetop_pos, pricetop_neg = get_top(price_df)

What people are saying about food:
0    food was not bad, but was really bad value for...
1              food was a tad mediocre for this place.
2                      food selection was not too bad.
3    the food in the colony is getting very boring ...
4    watching it change from the greenhouse to the ...
5    even though there were no dedicated kids secti...
6    contracted food poisoning the next day resulti...
7    there were some area is disappointment which i...
8    sadly it wasn't part of it, so i walked away t...
9         a lot of local food, not much international.
Name: review, dtype: object
What people are saying about service:
0        inconsistent service standards at the colony.
1               the service was lacking, sorry to say.
2    aside from that, the service staff went about ...
3    except for one chef at the local roast meat na...
4    we were appalled at the insistence by the wait...
5    we did not make a big fuss about it, but payin...
6    instead of tur

# 3. Name Entity 


In [15]:
# nlp_ent = spacy.load('en_core_web_sm',disable=['parser','tagger'])
# print(nlp_ent.pipeline)

# def to_entity(df, ent_name):

#   index = list()
#   for idx, doc in enumerate(nlp_ent.pipe(df['review'].to_list())):
#       for ent in doc.ents:
#           if ent.label_==ent_name:
#             index.append(idx)
#             print(doc.text)
#             print(idx, ent.text, ent.label_)
#             print('_________________')
#   entity = df.iloc[index].drop_duplicates()
#   entity.to_csv(f'{ent_name}.csv')
#   ent_pos, ent_neg = num_polarity(entity)

#   return entity, ent_pos, ent_neg

[('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x7f54c3d53648>)]


In [0]:
# entity, ent_pos, ent_neg = to_entity(main,'MONEY')

In [0]:
# # group by version 

# def to_entity(df, ent_name):

#   index = list()
#   group = list()
#   for idx, doc in enumerate(nlp_ent.pipe(df['review'].to_list())):
#       for ent in doc.ents:
#           if ent.label_==ent_name:
#             print(doc.text)
#             index.append(idx)
#             group.append(ent.text)
#             print(idx, ent.text, ent.label_)
#             print('_________________')
#   entity = df.iloc[index].drop_duplicates()
#   entity.to_csv(f'{ent_name}.csv')
#   ent_pos, ent_neg = num_polarity(entity)

#   return entity, group, ent_pos, ent_neg


In [0]:
# entity, group, ent_pos, ent_neg = to_entity(main,'NORP')
